## 2.1 The Unicode Standard

(a) What Unicode character does chr(0) return?



In [ ]:
c = chr(0)
print(c)          # nothing visible

chr(0) returns the Unicode character null

(b) How does this character's string representation ```(__repr__())``` differ from its printed representation?

In [ ]:
print(c.__repr__())

it shows an escaped form (```'\x00'```), which makes the character visible


(c) What happens when this character occurs in text? It may be helpful to play around with the
following in your Python interpreter and see if it matches your expectations:

In [ ]:
chr(0)
print(chr(0))
s = "this is a test" + chr(0) + "string"
print(s)

In [ ]:
print(s.__repr__())

'this is a teststring' gets printed. However, there is still a null between test and string as shown above.

## 2.2 Unicode Encodings

(a) What are some reasons to prefer training our tokenizer on UTF-8 encoded bytes, rather than UTF-16 or UTF-32? It may be helpful to compare the output of these encodings for various input strings.

One of the reasons is space efficiency for English/ ASCII characters (e.g.: character "A" in UTF-8 is 1 byte, in UTF-16 - 2 bytes, in UTF-32 - 4 bytes). Since most large-scale corpora are predominantly ASCII, UTF-8 yields a much smaller byte sequence to learn patterns over.

(b) Consider the following (incorrect) function, which is intended to decode a UTF-8 byte string into a Unicode string. Why is this function incorrect? Provide an example of an input byte string that yields incorrect results.

In [ ]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
  return "".join([bytes([b]).decode("utf-8") for b in bytestring])

decode_utf8_bytes_to_str_wrong("hello".encode("utf-8"))

The function is incorrect as it decodes each byte independently. That works for ASCII, however, as UTF-8 encodes some characters using more than 1 byte, hence it might not always work.

In [ ]:
decode_utf8_bytes_to_str_wrong("こ".encode("utf-8"))

In [ ]:
decode_utf8_bytes_to_str_wrong("🌍".encode("utf-8"))

## 2.5 Problem (train_bpe_tinystories): BPE Training on TinyStories (2 points)

(a) Train a byte-level BPE tokenizer on the TinyStories dataset, using a maximum vocabulary size of 10,000. Make sure to add the TinyStories <|endoftext|> special token to the vocabulary. Serialize the resulting vocabulary and merges to disk for further inspection.

How many hours and memory did training take? What is the longest token in the vocabulary? Does it make sense?

Resource requirements: ≤30 minutes (no GPUs), ≤30GB RAM Hint You should be able to get under 2 minutes for BPE training using multiprocessing during
pretokenization and the following two facts:

* (a) The <|endoftext|> token delimits documents in the data files.
* (b) The <|endoftext|> token is handled as a special case before the BPE merges are applied.



* Training time: 1029.17 seconds (17.15 minutes)
* Peak memory: 0.08 GB

Top longest tokens:
* ID 7159: ' accomplishment' (15 bytes)
* ID 9142: ' disappointment' (15 bytes)
* ID 9378: ' responsibility' (15 bytes)

Does it make sense? Yes. frequent word become a single token with a leading space per used regex.

(b) Profile your code. What part of the tokenizer training process takes the most time?

```

5725983 function calls (5725798 primitive calls) in 363.504 seconds

Ordered by: cumulative time
List reduced from 360 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      362  359.042    0.992  700.703    1.936 {built-in method time.sleep}
      3/2    0.000    0.000  363.536  181.768 /usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3512(run_code)
      3/2    0.000    0.000  363.536  181.768 {built-in method builtins.exec}
      109    0.000    0.000  341.941    3.137 /usr/lib/python3.12/multiprocessing/pool.py:500(_wait_for_updates)
      221    0.002    0.000  341.940    1.547 /usr/lib/python3.12/multiprocessing/connection.py:1122(wait)
      221    0.793    0.004  341.836    1.547 /usr/lib/python3.12/selectors.py:402(select)
        1    0.000    0.000  340.641  340.641 /usr/lib/python3.12/multiprocessing/pool.py:738(__exit__)
        9    0.000    0.000  340.592   37.844 /usr/lib/python3.12/multiprocessing/util.py:208(__call__)
        1    0.000    0.000  340.592  340.592 /usr/lib/python3.12/multiprocessing/pool.py:654(terminate)
        1    0.000    0.000  340.592  340.592 /usr/lib/python3.12/multiprocessing/pool.py:680(_terminate_pool)
        1    0.004    0.004  340.542  340.542 /usr/lib/python3.12/multiprocessing/pool.py:671(_help_stuff_finish)
        1    0.004    0.004  340.539  340.539 {method 'acquire' of '_multiprocessing.SemLock' objects}
      3/1    0.000    0.000  340.535  340.535 /usr/lib/python3.12/threading.py:1018(_bootstrap)
       16    0.000    0.000  340.535   21.283 /usr/lib/python3.12/multiprocessing/connection.py:202(send)
      3/1    0.000    0.000  340.535  340.535 /usr/lib/python3.12/threading.py:1058(_bootstrap_inner)
      3/1    0.000    0.000  340.535  340.535 /usr/lib/python3.12/threading.py:1001(run)
        1    0.000    0.000  340.534  340.534 /usr/lib/python3.12/multiprocessing/pool.py:527(_handle_tasks)
       21    0.000    0.000  340.534   16.216 /usr/lib/python3.12/multiprocessing/connection.py:406(_send_bytes)
       21    0.000    0.000  340.534   16.216 /usr/lib/python3.12/multiprocessing/connection.py:381(_send)
       21    0.000    0.000  340.534   16.216 {built-in method posix.write}
```

The slowest part is the parallel pretokenization pipeline and process synchronization (main process waiting for workers), rather than the core BPE training loop.

## Problem (train_bpe_expts_owt): BPE Training on OpenWebText (2 points)
(a) Train a byte-level BPE tokenizer on the OpenWebText dataset, using a maximum vocabulary size of 32,000. Serialize the resulting vocabulary and merges to disk for further inspection.

* What is the longest token in the vocabulary?
* Does it make sense?

Resource requirements: ≤12 hours (no GPUs), ≤100GB RAM



In [ ]:
import pickle, os

vocab_path = os.path.join(os.path.dirname(__file__) if "__file__" in dir() else ".", "..", "outputs", "owt_vocab_32k.pkl")
# Adjust path as needed for your environment
with open(vocab_path, "rb") as f:
    owt_vocab = pickle.load(f)

longest_id, longest_token = max(owt_vocab.items(), key=lambda x: len(x[1]))
try:
    decoded = longest_token.decode("utf-8")
except UnicodeDecodeError:
    decoded = repr(longest_token)

print(f"Longest token: {decoded!r}")

The longest token is a string ('ÃÂÃÂ...'); 
Should make sense as encoding artifacts are common in OpenWebText and BPE aggressively merges frequent repeated patterns.

(b) Compare and contrast the tokenizer that you get training on TinyStories versus OpenWebText.

The TinyStories tokenizer learns simple vocabulary (' accomplishment', ' disappointment') while the OWT tokenizer reflects web text including encoding artifacts, separator lines, URL fragments (http, www), and political and technical terms (' unconstitutional', ' cryptocurrencies').

## 2.7(a) Tokenizer Experiments — Compression Ratio

(A) Sample 10 documents from TinyStories and OpenWebText. Using your previously-trained TinyS-
tories and OpenWebText tokenizers (10K and 32K vocabulary size, respectively), encode these
sampled documents into integer IDs. What is each tokenizer's compression ratio (bytes/token)?

Average compression ratios (bytes/token) over 10 sampled documents:
TinyStories (10K vocab): 4.25 bytes/token
OpenWebText (32K vocab): 4.54 bytes/token

The difference is due to vocab size

(B) What happens if you tokenize your OpenWebText sample with the TinyStories tokenizer? Com-
pare the compression ratio and/or qualitatively describe what happens.

Tokenizing OWT text with the TinyStories tokenizer yields a compression ratio of 3.22 bytes/token (vs 4.54
with the native OWT tokenizer). The TinyStories vocabulary doesn't merged tokens for technical, political, and web-specific terms common in OWT, so the tokenizer falls back to shorter or single-byte tokens, producing more tokens for the same text

(C) Throughput: ~0.67 MB/s (measured on a 1 MB OWT sample). At that rate, tokenizing The Pile (825 GB) would take approximately 342 hours (~14 days).

(D) Using your TinyStories and OpenWebText tokenizers, encode the respective training and development datasets into a sequence of integer token IDs. We'll use this later to train our language
model. We recommend serializing the token IDs as a NumPy array of datatype uint16. Why is
uint16 an appropriate choice?

Why `uint16`? It stores values 0–65,535, which covers both vocab sizes and is the smallest type that fits the vocab sizes, minimizing storage and memory usage.

## 4.2 Problem (learning_rate_tuning): Tuning the learning rate (1 point)

Run the SGD toy example (Eq. 20: θ_{t+1} = θ_t − α/√(t+1) · ∇L) with lr ∈ {1e1, 1e2, 1e3} for 10 steps.

In [ ]:
import torch
import math
from typing import Optional
from collections.abc import Callable


class SGD(torch.optim.Optimizer):
    """SGD with decaying learning rate: θ_{t+1} = θ_t - (α / √(t+1)) · ∇L(θ_t; B_t)"""

    def __init__(self, params, lr=1e-3):
        if lr < 0:
            raise ValueError(f"Invalid learning rate: {lr}")
        defaults = {"lr": lr}
        super().__init__(params, defaults)

    def step(self, closure: Optional[Callable] = None):
        loss = None if closure is None else closure()
        for group in self.param_groups:
            lr = group["lr"]
            for p in group["params"]:
                if p.grad is None:
                    continue
                state = self.state[p]
                t = state.get("t", 0)
                grad = p.grad.data
                p.data -= lr / math.sqrt(t + 1) * grad
                state["t"] = t + 1
        return loss


learning_rates = [1e1, 1e2, 1e3]
results = {}

for lr in learning_rates:
    torch.manual_seed(42)
    weights = torch.nn.Parameter(5 * torch.randn((10, 10)))
    opt = SGD([weights], lr=lr)
    losses = []
    for t in range(10):
        opt.zero_grad()
        loss = (weights**2).mean()
        losses.append(loss.cpu().item())
        loss.backward()
        opt.step()
    results[lr] = losses

print(f"{'Step':>4}", end="")
for lr in learning_rates:
    print(f"  {'lr='+str(lr):>12}", end="")
print()
print("-" * 44)
for t in range(10):
    print(f"{t:>4}", end="")
    for lr in learning_rates:
        print(f"  {results[lr][t]:>12.4f}", end="")
    print()

- **lr=10**: Loss decays steadily but slowly (24.2 → 3.2 after 10 steps) — conservative step size, stable but gradual progress.
- **lr=100**: Loss converges rapidly to near zero by step 4 — the sweet spot where the optimizer takes big enough steps to converge quickly without overshooting.
- **lr=1000**: Loss explodes immediately (24.2 → 8,725 after 1 step, then to 10¹⁸) — the learning rate massively overshoots the minimum, causing divergence.

## 4.1.1 Problem (adamw): Implement AdamW (2 points)

Implemented in `ece496b_basics/optimizer.py`. Subclasses `torch.optim.Optimizer`.

`step` implements Algorithm 1:
1. `m ← β₁·m + (1−β₁)·g` — first moment update
2. `v ← β₂·v + (1−β₂)·g²` — second moment update
3. Bias-correct: `m̂ = m/(1−β₁ᵗ)`, `v̂ = v/(1−β₂ᵗ)`
4. `θ ← θ − α · m̂/(√v̂ + ε)` — parameter update
5. `θ ← θ − α·λ·θ` — decoupled weight decay

In [ ]:
import torch
import math
from torch.optim import Optimizer


class AdamW(Optimizer):
    """AdamW optimizer — Adam with decoupled weight decay."""

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.01):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        super().__init__(params, defaults)

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            lr = group["lr"]
            beta1, beta2 = group["betas"]
            eps = group["eps"]
            weight_decay = group["weight_decay"]

            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad
                state = self.state[p]
                if len(state) == 0:
                    state["step"] = 0
                    state["m"] = torch.zeros_like(p)
                    state["v"] = torch.zeros_like(p)

                state["step"] += 1
                t = state["step"]
                m, v = state["m"], state["v"]

                # Update biased moment estimates
                m.mul_(beta1).add_(grad, alpha=1 - beta1)
                v.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                # Bias correction
                m_hat = m / (1 - beta1 ** t)
                v_hat = v / (1 - beta2 ** t)

                # Adam update
                p.add_(m_hat / (v_hat.sqrt() + eps), alpha=-lr)

                # Decoupled weight decay
                if weight_decay != 0:
                    p.add_(p, alpha=-lr * weight_decay)

        return loss


# Quick sanity check
torch.manual_seed(0)
model = torch.nn.Linear(3, 2)
opt = AdamW(model.parameters(), lr=1e-2, weight_decay=0.01)
for _ in range(5):
    opt.zero_grad()
    loss = (model(torch.randn(4, 3)) ** 2).mean()
    loss.backward()
    opt.step()
print(f"AdamW sanity check — final loss: {loss.item():.4f}")

## 4.3 Problem (adamwAccounting): Resource accounting with AdamW (2 points)

**(a)** Peak memory expressions (float32 = 4 bytes per value):

Let B = batch_size, S = context_length, D = d_model, H = num_heads, L = num_layers, V = vocab_size, F = d_ff = 4D.

**Parameters (P):**
Per block: 4D² (QKV+O) + 2·D·F (FFN W1,W2) + 2D (two RMSNorms) = 12D² + 2D
Full model: **P = L·(12D² + 2D) + 2VD + D**
Memory: 4P bytes

**Gradients:** 4P bytes
**Optimizer state (AdamW):** m + v = 8P bytes

**Activations** (stored for backward, per block ×L):
- RMSNorm inputs (×2): 2BSD
- Q, K, V projections: 3BSD
- Softmax output: BHS²
- Attention output + residuals + FFN intermediates: 11BSD

Per-block total: **16BSD + BHS²**
Model-level: 2BSD + 2BSV

**Total peak memory = 16P + 4·BS·[L·(16D + HS) + 2D + 2V] bytes**

**(b)** Max batch size for GPT-2 XL on 80 GB A100:

GPT-2 XL: L=48, D=1600, H=25, S=1024, V=50257, F=6400.

P = 48·(12·1600² + 2·1600) + 2·50257·1600 + 1600 = **1.64B parameters**
Fixed cost = 16P = **26.17 GB**

Activation cost per sample ≈ **10.49 GB**
Memory (GB) ≈ 10.49·B + 26.17

80 ≥ 10.49·B + 26.17 → B ≤ 5.13 → **B_max = 5**

**(c)** AdamW FLOPs per step:

Per parameter: ~15 FLOPs (3 for m update, 4 for v update, 2 for bias corrections, 4 for Adam update, 2 for weight decay).

For GPT-2 XL: 15 × 1.64B ≈ **24.5 GFLOPs** per optimizer step

**(d)** GPT-2 XL training time on a single A100 (B=1024, 400K steps, 50% MFU):

Forward FLOPs per step (B=1024): **3,591 TFLOPs**
Backward ≈ 2× forward: 7,182 TFLOPs
Total per step: **10,773 TFLOPs**

A100 at 50% MFU: 19.5 × 0.5 = 9.75 TFLOP/s
Time per step: 10,773 / 9.75 = 1,105 seconds
Total: 1,105 × 400,000 = **5,116 days ≈ 14.0 years**

Completely impractical on a single GPU. Real training uses hundreds of GPUs with data/tensor parallelism.